In [1]:
import pandas as pd
df = pd.read_csv("inference_data_indices.csv")
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')

In [2]:
badgers = pd.read_csv("Meles meles.csv", delimiter='\t')
badgers = badgers[badgers["occurrenceStatus"] == "PRESENT"]
badgers = badgers[badgers['year']>=2022]
badgers['eventDate'] = pd.to_datetime(badgers['eventDate'], format='%Y-%m-%d', errors='coerce')
badgers = badgers[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
badgers = badgers.dropna().reset_index(drop=True)

ground_beetles = pd.read_csv("ground_beetles.csv", delimiter='\t')
ground_beetles = ground_beetles[ground_beetles["occurrenceStatus"] == "PRESENT"]
ground_beetles = ground_beetles[ground_beetles['year']>=2022]
ground_beetles['eventDate'] = pd.to_datetime(ground_beetles['eventDate'], format='%Y-%m-%d', errors='coerce')
ground_beetles = ground_beetles[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
ground_beetles = ground_beetles.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_80504/1507747312.py:1: DtypeWarning: Columns (10,16,39,41,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  badgers = pd.read_csv("Meles meles.csv", delimiter='\t')
/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_80504/1507747312.py:8: DtypeWarning: Columns (2,10,36,38,39,41,45,48) have mixed types. Specify dtype option on import or set low_memory=False.
  ground_beetles = pd.read_csv("ground_beetles.csv", delimiter='\t')


In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from tqdm import tqdm

# Convert lat/lon to radians for BallTree
hedgehog_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
badger_coords = np.deg2rad(badgers[['decimalLatitude', 'decimalLongitude']].values)
ground_beetles_coords = np.deg2rad(ground_beetles[['decimalLatitude', 'decimalLongitude']].values)

# Build BallTrees
tree_badger = BallTree(badger_coords, metric='haversine')
tree_ground_beetles = BallTree(ground_beetles_coords, metric='haversine')

# 1 km radius in radians
radius = 1 / 6371.0

# Initialize presence columns
df['badger_presence'] = 0
df['ground_beetles_presence'] = 0

# Iterate through each hedgehog point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = hedgehog_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # BADGERS
    idxs = tree_badger.query_radius(point, r=radius)[0]
    for j in idxs:
        badger_date = badgers.loc[j, 'eventDate']
        if badger_date.month == event_month and badger_date.year == event_year:
            df.at[i, 'badger_presence'] = 1
            break  # Found at least one match, no need to check further

    # GROUND BEETLES
    idxs = tree_ground_beetles.query_radius(point, r=radius)[0]
    for j in idxs:
        beetle_date = ground_beetles.loc[j, 'eventDate']
        if beetle_date.month == event_month and beetle_date.year == event_year:
            df.at[i, 'ground_beetles_presence'] = 1
            break

100%|██████████| 244359/244359 [00:27<00:00, 8832.83it/s] 


In [ ]:
# import osmnx as ox
# from shapely.geometry import Point
# import geopandas as gpd
# import numpy as np
# from tqdm import tqdm
# 
# # Construct GeoDataFrame
# df['geometry'] = [Point(xy) for xy in zip(df['decimalLongitude'], df['decimalLatitude'])]
# gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
# 
# # Define spatial tiling: 0.05 x 0.05 degrees
# tile_size = 0.1
# gdf['tile_x'] = (gdf['decimalLongitude'] // tile_size).astype(int)
# gdf['tile_y'] = (gdf['decimalLatitude'] // tile_size).astype(int)
# 
# # Group by tile
# grouped = gdf.groupby(['tile_x', 'tile_y'])
# 
# # Store results
# all_distances = []
# 
# for (tile_x, tile_y), group in tqdm(grouped, total=len(grouped)):
#     west = tile_x * tile_size - 0.02
#     south = tile_y * tile_size - 0.02
#     east = (tile_x + 1) * tile_size + 0.02
#     north = (tile_y + 1) * tile_size + 0.02
#     bbox = (west, south, east, north)
# 
#     try:
#         G = ox.graph_from_bbox(bbox, network_type='drive_service')
#         if len(G.nodes) == 0:
#             print(f"[EMPTY GRAPH] bbox: {bbox}, skipping...")
#             all_distances.extend([np.nan] * len(group))
#             continue
# 
#         G_proj = ox.project_graph(G)
#         nodes_proj, edges_proj = ox.graph_to_gdfs(G_proj)
#         points_proj = group.geometry.to_crs(nodes_proj.crs)
# 
#         for point_proj in points_proj:
#             try:
#                 u, v, k = ox.distance.nearest_edges(G_proj, [point_proj.x], [point_proj.y])[0]
#                 edge_geom = edges_proj.loc[(u, v, k)]['geometry']
#                 distance = point_proj.distance(edge_geom)
#                 all_distances.append(distance)
#             except Exception as e:
#                 print(f"  [Point ERROR] {e}")
#                 all_distances.append(np.nan)
# 
#     except Exception as e:
#         print(f"[TILE ERROR] {e} — bbox: {bbox}")
#         all_distances.extend([np.nan] * len(group))
# 
# 
# # Store back in original DataFrame order
# gdf['distance_to_road'] = all_distances

In [ ]:
# gdf['near_road'] = np.where(gdf['distance_to_road'] <= 500, 1, 0)        

In [ ]:
# gdf=gdf.drop(['tile_x', 'tile_y', 'geometry', 'distance_to_road'], axis=1)

In [ ]:
# from pyproj import Transformer
# from rasterio.windows import Window
# import rasterio
# 
# land_cover_map = {
#     1: "Deciduous woodland",
#     2: "Coniferous woodland",
#     3: "Arable",
#     4: "Improved grassland",
#     5: "Neutral grassland",
#     6: "Calcareous grassland",
#     7: "Acid grassland",
#     8: "Fen",
#     9: "Heather",
#     10: "Heather grassland",
#     11: "Bog",
#     12: "Inland rock",
#     13: "Saltwater",
#     14: "Freshwater",
#     15: "Supralittoral rock",
#     16: "Supralittoral sediment",
#     17: "Littoral rock",
#     18: "Littoral sediment",
#     19: "Saltmarsh",
#     20: "Urban",
#     21: "Suburban"
# }
# 
# # Batch coordinate transformation
# transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
# transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)
# 
# coords = list(zip(gdf['decimalLongitude'], gdf['decimalLatitude']))
# gdf['easting_ni'], gdf['northing_ni'] = zip(*transformer_ni.itransform(coords))
# gdf['easting_gb'], gdf['northing_gb'] = zip(*transformer_gb.itransform(coords))
# 
# # Raster processing optimization
# gb_raster = 'gblcm2023_10m.tif'
# n_ireland_raster = 'nilcm2023_10m.tif'
# 
# 
# def get_land_cover_class(row):
#     try:
#         # Try GB raster first
#         with rasterio.open(gb_raster) as src:
#             row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])
#             # Read a small window around the point for better performance
#             window = Window(col_idx, row_idx, 1, 1)
#             land_cover_class = src.read(1, window=window)[0, 0]
# 
#             if land_cover_class == 0:  # Check NI raster if GB is 0
#                 with rasterio.open(n_ireland_raster) as src_ni:
#                     row_idx, col_idx = src_ni.index(row['easting_ni'], row['northing_ni'])
#                     window = Window(col_idx, row_idx, 1, 1)
#                     land_cover_class = src_ni.read(1, window=window)[0, 0]
# 
#         return land_cover_map.get(land_cover_class, "Unknown")
#     except Exception as e:
#         print(f"Error processing row: {e}")
#         return "Unknown"
# 
# 
# tqdm.pandas()
# gdf['Land_cover'] = gdf.progress_apply(get_land_cover_class, axis=1)


In [ ]:
gdf = gdf.dropna()
gdf=gdf.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
gdf = gdf[gdf['Land_cover'] != "Unknown"].reset_index(drop=True)
gdf.to_csv("hedgehog_inference_data_preprocessed.csv", index=False)

In [ ]:
gdf = gdf.dropna()
gdf = gdf.drop_duplicates()
gdf = gdf.drop(columns=['species'])
hedgehog_bool_columns = ['badger_presence','ground_beetles_presence','near_road']
gdf[hedgehog_bool_columns] = gdf[hedgehog_bool_columns].astype(bool)
gdf = pd.get_dummies(gdf, columns=['Land_cover'])
gdf.to_csv("hedgehog_inference_data_preprocessed.csv")

In [ ]:
gdf = pd.read_csv("hedgehog_inference_data_preprocessed.csv")
gdf = gdf.astype({col: 'int' for col in gdf.select_dtypes(include='bool').columns})

#read scaler
import joblib
scaler = joblib.load("hedgehog_scaler.pkl")
# Scale the data
env_vars = ['BSI', 'LST', 'MNDWI', 'NDBI', 'NDSI', 'NDVI', 'NDWI', 'SAVI', 'UI']
gdf[env_vars] = scaler.transform(gdf[env_vars])
# drop Unnamed: 0
# df = df.drop(columns=['Unnamed: 0'])
df